In [211]:
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings('ignore')

In [212]:
Data2 = pd.read_excel('/Users/briankimanzi/Documents/programming Languages/PythonProgramming/JupyterNoteBook/Datasets/MPESA_Marylyne.xlsx')

In [213]:
# Convert to datetime (handles mixed formats)
Data2['Completion Time'] = pd.to_datetime(Data2['Completion Time'], errors='coerce')

# Remove microseconds
Data2['Completion Time'] = Data2['Completion Time'].dt.floor('s')  # Truncate to seconds

In [62]:
Data2['Completion Time'].

713

In [234]:
# Anonymising the dataset 
def transaction(x):
    x = str(x).strip()
    if x.startswith('Merchant Payment'):
        index = x.find(' - ')+2
        name = x[index:].strip().upper()
        
        return 'BUY GOODS', name

    elif x.startswith('Deposit of Funds'):
        index = x.find(' - ')+2
        name = x[index:].strip().upper()
        
        return 'AGENT DEPOSIT', name

    elif x.startswith('OD Loan Repayment'):
        return 'FULIZA REPAYMENT', 'FULIZA'

    elif x.startswith('OverDraft of Credit Party'):
        return 'FULIZA TAKEN', 'FULIZA'

    elif x.startswith('M-Shwari Deposit'):
        return 'M-SHWARI DEPOSIT FROM M-PESA', 'M-SHWARI'
        
    elif x.startswith('KCB M-PESA Deposit'):
        return 'KCB M-PESA DEPOSIT FROM M-PESA', 'KCB DEPOSIT'

    elif x.startswith('KCB M-PESA Withdraw'):
        return 'KCB M-PESA WITHDRAW FROM M-PESA', 'KCB WITHDRAW'

    elif x.startswith('Customer Transfer'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
                
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        return 'SEND MONEY', name

    elif x.startswith('M-Shwari Withdraw'):
        
        return 'M-SHWARI WITHDRAW FROM M-PESA', 'M-SHWARI WITHDRAW'

    elif x.startswith('Pay Bill Charge'):
        # if x.strip() == 'Pay Bill Charge':
            return 'PAY BILL CHARGES', 'TRANSACTION COST'
        
    elif x.startswith('Pay Bill Online to'):
        if x == 'Pay Bill Online to':  
            return 'PAY BILL ONLINE', 'TRANSACTION COST'
        else:
            start = len('Pay Bill Online to')+9
            end = x.lower().find('acc')
            name = x[start:end].strip().upper()
            return 'PAY BILL', name
    elif x.startswith('Pay Bill'):
        if x == 'Pay Bill':  
            return 'PAY BILL', 'UNKNOWN'
        else:
            start = x.find(' - ') + 3 if ' - ' in x else len('Pay Bill') + 1
            end = x.lower().find('acc')
            name = x[start:end].strip().upper() if end != -1 else x[start:].strip().upper()
            return 'PAY BILL', name

    elif x.startswith('Funds received'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        
        return 'RECEIVED FUNDS', name

    elif x.startswith('Customer Payment to Small Business') or x.startswith('Customer Send Money'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        
        return 'POCHI LA BIASHARA', name

    elif x.startswith('Airtime Purchase'):
        return 'AIRTIME PURCHASE', 'AIRTIME'

    elif x.startswith('Business Payment From'):
        index = x.find(' - ')
        end = x.lower().find('via')
        to_search = x[index:]
        last = 1
        name = x[index:end].strip().upper()
        return 'FUNDS RECEIVED FROM BUSINESS', name

    elif x.startswith('Customer Transfer of Funds Charge'):
        return 'TRANSACTION COST', 'TRANSACTION COST'
    
    elif x.startswith('Buy Bundles Online'):
        return 'BUNDLES PURCHASE', 'BUNDES PURCHASE'
    
    elif x.startswith('Customer Withdrawal'):
        index = x.find(' - ')+2
        name = x[index:].strip().upper()
        return 'CASH WITHDRAWAL', name
    
    elif x.startswith('Withdrawal Charge'):
        return 'CASH WITHDRAWAL CHARGES', "TRANSACTION COST"

    elif x.startswith('Customer Bundle Purchase') or x.startswith('Recharge for Customer'):
        return 'CUSTOMER BUNDLE PURCHASE', 'SAFARICOM DATA BUNDLES'

    elif x.startswith('Promotion Payment'):
        return 'PROMOTION PAYMENT', 'M-PESA OFFERS. VIA API'
    
    elif x.startswith('Business Payment'):
        return 'BUSINESS PAYMENT', 'CO-OP BANK VIA API'
        
    elif x.startswith('Receive International'):
        index = x.find(' - ')+3
        remaining_str = x[index:]
        name = remaining_str.split('.')[0].strip().upper()
        return 'RECEIVE INTERNATION TRANSFER', name
        
    elif x.startswith('Send Money Reversal'):
        index = x.find(' - ')+17
        name = x[index:]
        return 'SEND MONEY REVERSAL', name
        
    elif x.startswith('Savings Contribution'):
        return 'TO HUSTLER FUND SAVINGS', 'HUSTLER FUND'
        
    elif x.startswith('OverDraft  of Credit Party'):
        return 'OVERDRAFT', 'CREDIT PARTY'

    elif x.startswith('714888'):
        index = x.find(' - ')+2
        remaining_str = x[index:]
        name = remaining_str.split('.')[0].strip().upper()
        return 'ACCOUNT PAYMENT', name
    
    elif x.startswith('Term Loan Disbursement for H- Fund') or x.startswith('Term Loan Disbursement for H-Fund'):
        return 'HUSTLER FUND Disbursement'.upper(), 'HUSTLER FUND'
    
    elif x.startswith('Term Loan Repayment for H- Fund') or x.startswith('Term Loan Repayment for H-Fund'):
        return 'HUSTLER FUND REPAYMENT', 'HUSTLER FUND'
    
    else:
        return 'UNIDENTIFIED', 'UNIDENTIFIED'

In [235]:
details = list(Data2['Details'].apply(lambda x: transaction(x)).values)
transactionType = [i[0] for i in details]
TransactionParty = [i[1] for i in details]

In [236]:
# changing the date format
def change_date(date):
    date_str = str(date)
    if pd.isna(date):
        return (None, None, None, None, None, None, None)
    date = parser.parse(date_str)
    return (date.year, date.month, date.day, date.weekday(),date.hour, date.minute, date.second)

In [237]:
date = list(Data2['Completion Time'].apply(lambda x: change_date(x)).values)
Year = [i[0] for i in date]
Month = [i[1] for i in date]
Date = [i[2] for i in date]
Weekday = [i[3] for i in date]
Hour = [i[4] for i in date]
Minute = [i[5] for i in date]
Seconds = [i[6] for i in date]

transactionDay = []

reverseDay = Date.copy()[::-1]
datey = reverseDay[0]

d = 1

for i in reverseDay:
    if i == datey:
        transactionDay.append(d)
    else:
        datey = i
        d += 1
        transactionDay.append(d)

transactionDay.reverse()

In [238]:
receipt = list(Data2['Receipt No.'].values)

In [239]:
paid_in = Data2['Paid In'].fillna('NAN_VALUE').values

In [240]:
def withdrawAmount(x):
    try:
        if x < 0:
            return -x
        else:
            return x
    except:
        return x

In [241]:
for col in ['Paid In', 'Withdrawn', 'Balance']:
    # Remove commas, dollar signs, and strip whitespace
    Data2[col] = Data2[col].astype(str).str.replace('[^\d.-]', '', regex=True)
    
    # Convert empty strings to NaN
    Data2[col] = Data2[col].replace('', np.nan)
    
    # Now convert to float
    Data2[col] = pd.to_numeric(Data2[col], errors='coerce')  # Invalid parsing → NaN

In [242]:
withdraw = list(Data2['Withdrawn'].apply(lambda x: withdrawAmount(x)).values)
withdraw                

[65.0,
 120.0,
 70.0,
 57.0,
 10500.0,
 160.0,
 10.0,
 50.0,
 60.0,
 10.0,
 800.0,
 70.0,
 70.0,
 45.0,
 7.0,
 150.0,
 3006.0,
 100.0,
 70.0,
 90.0,
 5.0,
 400.0,
 65.0,
 190.0,
 30.0,
 80.0,
 70.0,
 7.0,
 500.0,
 7.0,
 500.0,
 585.0,
 890.25,
 nan,
 45.0,
 nan,
 75.0,
 nan,
 65.0,
 nan,
 200.0,
 nan,
 75.0,
 nan,
 7.0,
 500.0,
 nan,
 nan,
 300.0,
 30.0,
 nan,
 65.0,
 nan,
 130.0,
 nan,
 nan,
 500.0,
 75.0,
 nan,
 13.0,
 900.0,
 nan,
 nan,
 388.56,
 90.0,
 nan,
 70.0,
 nan,
 60.0,
 nan,
 70.0,
 nan,
 60.0,
 nan,
 65.0,
 nan,
 11.0,
 100.0,
 40.0,
 25.0,
 40.0,
 75.0,
 65.0,
 190.0,
 75.0,
 45.0,
 65.0,
 70.0,
 30.0,
 30.0,
 100.0,
 70.0,
 nan,
 75.0,
 65.0,
 190.0,
 100.0,
 5.0,
 350.0,
 50.0,
 103.71,
 nan,
 141.0,
 nan,
 45.0,
 70.0,
 65.0,
 65.0,
 150.0,
 7.0,
 250.0,
 50.0,
 200.0,
 1530.0,
 50.0,
 160.0,
 45.0,
 75.0,
 nan,
 65.0,
 190.0,
 45.0,
 10.0,
 28.0,
 70.0,
 28.0,
 8.0,
 7.0,
 145.0,
 5.0,
 150.0,
 10.0,
 28.0,
 75.0,
 80.0,
 65.0,
 10.0,
 150.0,
 nan,
 48.0,
 75.0,
 70.0

In [243]:
transaction_Data = []

for i in range(len(paid_in)):
    try:
        x = float(paid_in[i])
        transaction_Data.append((x, 'PAID IN'))
    except:
        x = float(withdraw[i])
        transaction_Data.append((x, 'WITHDRAW'))

In [244]:
transaction_amount = [i[0] for i in transaction_Data]
paid_in_or_withdraw = [i[1] for i in transaction_Data]

In [245]:
balance =list(Data2["Balance"].values)

In [253]:
Nashipae = pd.DataFrame({
    "Receipt":receipt,
    "transaction_Day": transactionDay,
    "Year" : Year,
    "Month" : Month,
    "Date" : Date,
    "Weekday" : Weekday,
    "Hour" : Hour,
    "Minute" : Minute,
    "Seconds" : Seconds,
    "Transaction_type" : transactionType,
    "Transaction_party" : TransactionParty,
    "Transaction_amount" : transaction_amount,
    "paid_in_or_Withdraw" : paid_in_or_withdraw,
    "Balance" : balance
})
Nashipae.set_index('Receipt', inplace=True)

In [249]:
Nashipae.reset_index(inplace=True)

In [ ]:
Nashipae[Nashipae['Transaction_type'] == 'ACCOUNT PAYMENT']

In [254]:
Nashipae

,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
Receipt,,,,,,,,,,,,,
TD349XUJFS,154,2025,4,3,3,18,27,59,BUY GOODS,SAWA SHOP 4,65.00,WITHDRAW,1260.75
TD399XA2KP,154,2025,4,3,3,18,26,0,BUY GOODS,MOHAMED DULACHA ALI,120.00,WITHDRAW,1325.75
TD359WGPPX,154,2025,4,3,3,18,22,0,BUY GOODS,JOANINA KARIMI RUKARIA 6,70.00,WITHDRAW,1445.75
TD359Q25IZ,154,2025,4,3,3,17,53,0,PAY BILL CHARGES,TRANSACTION COST,57.00,WITHDRAW,1515.75
TD359Q25IZ,154,2025,4,3,3,17,53,0,PAY BILL,CO- OPERATIVE BANK MONEY TRANSFER,10500.00,WITHDRAW,1572.75
TD339NKBL9,154,2025,4,3,3,17,41,0,BUY GOODS,IBRAHIM NGUGI 2,160.00,WITHDRAW,12072.75
TD309I3OBK,154,2025,4,3,3,17,12,59,SEND MONEY,ALICE WANJOHI,10.00,WITHDRAW,12232.75
TD3986S50H,154,2025,4,3,3,12,24,59,BUY GOODS,GIVE AND SHARE FAST FOODS,50.00,WITHDRAW,12242.75
TD3786IFAF,154,2025,4,3,3,12,22,59,BUY GOODS,HANI WARIO GUYO,60.00,WITHDRAW,12292.75


In [247]:
Nashipae = Nashipae.astype({
    'Year': 'int64',
    'Month': 'int64',
    'Date': 'int64',
    'Weekday': 'int64',
    'Hour': 'int64',
    'Minute': 'int64',
    'Seconds': 'int64'
}, errors='ignore').fillna({
    'Year': 0,
    'Month': 0,
    'Date': 0,
    'Weekday': 0,
    'Hour': 0,
    'Minute': 0,
    'Seconds': 0
})

In [255]:
Nashipae.to_csv('/Users/briankimanzi/Documents/programming Languages/PythonProgramming/JupyterNoteBook/Datasets/Nashipae.csv')